In [111]:
import pandas as pd
import numpy as np
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---


In [112]:
#start_time = time.time()
#Phase1 Using the offline workload A as the traning data

#Step1 Load data

workload_A_all=pd.read_csv('../../offline_workload.csv') 
#print(workload_A.head())
tmp_flag = (workload_A_all != 0).any(axis=0)
workload_A = workload_A_all.loc[:, tmp_flag]
#workload_A
workload_A = workload_A.loc[:, ~(workload_A == workload_A.iloc[0]).all()]

workload_A = workload_A.loc[:, workload_A.isin([0]).mean() < .6]



#print(workload_A_all)
#print("--- %s seconds ---" % (time.time() - start_time))

In [113]:
# Normalization
from sklearn.preprocessing import MinMaxScaler

tmpframe = workload_A.T.copy()

tmpframe1 = tmpframe.tail(314-1).T
tmpframe2 = tmpframe.head(1).T

knobs_metrics_name = tmpframe1.columns
work_id_name = tmpframe2.columns

tmp_workload_knobs_metrics = (workload_A[knobs_metrics_name]).values

dataScalar = MinMaxScaler().fit(tmp_workload_knobs_metrics)

workload_knobs_metrics_Norm = dataScalar.transform(tmp_workload_knobs_metrics)
tmp_workload_id = (workload_A[work_id_name]).values

workload_knobs_metrics_Norm_id = np.hstack((tmp_workload_id,workload_knobs_metrics_Norm))

col_list = list(work_id_name)+list(knobs_metrics_name)
norm_workload_A = pd.DataFrame(data=workload_knobs_metrics_Norm_id,columns=col_list)


In [114]:
start_time = time.time()

#Step2 Split workload_A by creating different dataframes

workload_A_ids = norm_workload_A["workload id"].unique()
workload_A_list = []
for num,workload_A_id in enumerate(workload_A_ids,start=1):
    tmp_workload = norm_workload_A[norm_workload_A['workload id'].isin([workload_A_id])]
    workload_A_list.append(tmp_workload)
    exec("workload_A_%s = tmp_workload"%num)

#workload_A_1
print("--- %s seconds ---" % (time.time() - start_time))
#workload_A_list[1]

--- 0.1167140007019043 seconds ---


In [115]:
start_time = time.time()

#Step3 Get metrics

tmpframe = norm_workload_A.T.copy()
#tmpframe = tmpframe.tail(585-1-12).T
#tmpframe = tmpframe.tail(414-1-12).T
#tmpframe = tmpframe.tail(372-1-12).T
tmpframe = tmpframe.tail(314-1-12).T
metrics_name = tmpframe.columns
#print(metrics_name)
workload_A_listformetrics = []
num_workloads_A = len(workload_A_list)
for i in range(num_workloads_A):
    tmp = (workload_A_list[i])[metrics_name]
    workload_A_listformetrics.append(tmp)

print("--- %s seconds ---" % (time.time() - start_time))
(tmpframe)

--- 1.4870810508728027 seconds ---


,latency,mimic_cpu_util,driver.BlockManager.memory.maxMem_MB.avg,driver.BlockManager.memory.maxOnHeapMem_MB.avg,driver.BlockManager.memory.memUsed_MB.avg,driver.BlockManager.memory.onHeapMemUsed_MB.avg,driver.BlockManager.memory.remainingMem_MB.avg,driver.BlockManager.memory.remainingOnHeapMem_MB.avg,driver.DAGScheduler.job.activeJobs.avg,driver.DAGScheduler.job.allJobs.avg_increase,...,worker_2.Paging_and_Virtual_Memory.pgactivate,worker_2.Paging_and_Virtual_Memory.pgalloc_normal,worker_2.Paging_and_Virtual_Memory.pgfault,worker_2.Paging_and_Virtual_Memory.pgfree,worker_2.Paging_and_Virtual_Memory.pgpgin,worker_2.Paging_and_Virtual_Memory.pgpgout,worker_2.Processes.Blocked,worker_2.Processes.Runnable,worker_2.Processes.fork,worker_2.Processes.pswitch
0,0.0134949,0.0246714,0.016629,0.016629,0.000916009,0.000916009,0.0236648,0.0236648,0.677154,0.118812,...,0.0477606,0.048065,0.012964,0.0357318,0.000638225,0.0480704,0.0232684,0.0395533,0.0929657,0.065145
1,0.0009664,0.389111,0.783374,0.783374,0.00477507,0.00477507,0.78413,0.78413,0.221831,0.0594059,...,0.335875,0.292873,0.0780634,0.240153,0.00950681,0.313699,0.383333,0.351963,0.439727,0.207286
2,0.00168567,0.340512,0.387842,0.387842,0.0025399,0.0025399,0.391883,0.391883,0.433099,0.148515,...,0.184134,0.14993,0.0513811,0.124257,0.00940101,0.186962,0.105556,0.218565,0.376413,0.180328
3,0.00230275,0.262531,0.308037,0.308037,0.00252109,0.00252109,0.31266,0.31266,0.363682,0.178218,...,0.15389,0.116555,0.0415938,0.101355,0,0.172973,0.0972222,0.169665,0.310851,0.139158
4,0.00617735,0.0831844,0.072887,0.072887,0.00128761,0.00128761,0.0794478,0.0794478,0.586268,0.207921,...,0.140285,0.0765939,0.0193498,0.058595,0,0.12488,0.1875,0.0738212,0.171111,0.0924122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18344,0.056588,0.53047,0.748011,0.748011,0.0174607,0.0174607,0.746732,0.746732,0.782774,0.19802,...,0.297283,0.314484,0.490009,0.240323,0,0.623782,0.192389,0.475659,0.0407299,0.0571209
18345,0.0374673,0.791403,0.88653,0.88653,0.0152282,0.0152282,0.884647,0.884647,0.768306,0.207921,...,0.102397,0.165727,0.603045,0.123939,0,0.198524,0.0703551,0.715209,0.0550059,0.067304
18346,0.0631478,0.446528,0.410449,0.410449,0.00664394,0.00664394,0.413586,0.413586,0.794761,0.217822,...,0.273018,0.279591,0.431849,0.228121,0,0.600574,0.179327,0.335596,0.0341861,0.0477664
18347,0.080886,0.345435,0.313089,0.313089,0.00365573,0.00365573,0.317468,0.317468,0.800002,0.039604,...,0.255025,0.233936,0.37855,0.18332,0.00910968,0.54745,0.152694,0.261588,0.0265045,0.0464286


In [116]:
#Step4 FA for all workloads together
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import normalize


start_time = time.time()
all_metrics_data = tmpframe.values

all_metrics_data_Trans = all_metrics_data.T

#all_metrics_data_normalized = normalize(all_metrics_data, norm='l2')
#all_metrics_data_normalized_Trans = all_metrics_data_normalized.T

#all_metrics_data_Trans = all_metrics_data.T
#all_metrics_data_normalized_Trans = normalize(all_metrics_data_Trans, norm='max')

print(all_metrics_data_normalized_Trans.shape)
tmp_all_transformer = FactorAnalysis(n_components=50, random_state=0)
tmp_workload_A_transformed = tmp_all_transformer.fit_transform(all_metrics_data_Trans)
print(tmp_workload_A_transformed.shape)
print("--- %s seconds ---" % (time.time() - start_time))

(301, 18349)
(301, 50)
--- 2.5360543727874756 seconds ---


In [117]:
#Step5 K-means for all workloads together
from sklearn.cluster import KMeans

num_M = 3

workload_A_list_array_Kmeans = KMeans(n_clusters=num_M, random_state=0).fit(tmp_workload_A_transformed)
cluster_centers = workload_A_list_array_Kmeans.cluster_centers_
tmp_centers_index = []
tmp_centers_metrics_name = []
for i in range(num_M):
    tmp_center = cluster_centers[i]
    tmp_center_diff = tmp_center - tmp_workload_A_transformed
    tmp_center_distance = np.sum((tmp_center_diff * tmp_center_diff),axis=1)
    tmp_center_min_idx = np.argmin(tmp_center_distance)
    tmp_centers_index.append(tmp_center_min_idx)
    
for j in range(num_M):
        tmp_mname = metrics_name[tmp_centers_index[j]]
        #print(tmp_mname)
        tmp_centers_metrics_name.append(tmp_mname)
print(tmp_centers_index)
print(tmp_centers_metrics_name)
#tmp_centers_metrics_name.append('latency')
tmp_centers_metrics_name = ['latency'] + tmp_centers_metrics_name
print(tmp_centers_metrics_name)
num_M += 1
workload_list_centers_metrics_name = []
for n in range(58):
    workload_list_centers_metrics_name.append(tmp_centers_metrics_name)


[163, 178, 98]
['worker_1.Disk_Write_KB/s.dm-0', 'worker_1.JFS_Filespace_%Used./home', 'executor.jvm.non-heap.committed.nb_change']
['latency', 'worker_1.Disk_Write_KB/s.dm-0', 'worker_1.JFS_Filespace_%Used./home', 'executor.jvm.non-heap.committed.nb_change']


In [118]:
start_time = time.time()

#Step7 Building GPR (apply for all workloads)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler

tmp_kernel = 1.0 * RBF(length_scale=1.0)
knobs_list = ['k1','k2','k3','k4','k5','k6','k7','k8','s1','s2','s3','s4']

workload_A_GPRmodels_lists = []
workload_A_scalar_lists = []
workload_A_normalized_m = []

for i in range(58):
    tmp_GPRmodels_lists = []
    tmp_scalar_lists = []
    tmp_normalized_m = []
    
    tmp_Pruning_workload_A_X = ((workload_A_list[i])[knobs_list]).values
    #tmp_Pruning_workload_A_y_o = ((workload_A_list[i])[workload_list_centers_metrics_name]).values
    
    tmp_num_centers = len(tmp_centers_metrics_name)
    
    for j in range(tmp_num_centers):
        
        tmp_Pruning_workload_A_y_m = ((workload_A_list[i])[tmp_centers_metrics_name[j]]).values
        tmp_workload_A_gpr_m = GaussianProcessRegressor(kernel=tmp_kernel).fit(tmp_Pruning_workload_A_X, tmp_Pruning_workload_A_y_m)
        tmp_GPRmodels_lists.append(tmp_workload_A_gpr_m)
        
    workload_A_GPRmodels_lists.append(tmp_GPRmodels_lists)

print("--- %s seconds ---" % (time.time() - start_time))
#print(workload_A_normalized_m)

--- 31.7106351852417 seconds ---


In [119]:
start_time = time.time()

#Step1 load workload B data set

workload_B=pd.read_csv('../../online_workload_B.csv') 

#knobs_metrics_name = tmpframe1.columns
#work_id_name = tmpframe2.columns

tmp_workload_B_knobs_metrics = (workload_B[knobs_metrics_name]).values

#dataScalar = MinMaxScaler().fit(tmp_workload_knobs_metrics)

Bworkload_knobs_metrics_Norm = dataScalar.transform(tmp_workload_B_knobs_metrics)
Btmp_workload_id = (workload_B[work_id_name]).values

Bworkload_knobs_metrics_Norm_id = np.hstack((Btmp_workload_id,Bworkload_knobs_metrics_Norm))

#col_list = list(work_id_name)+list(knobs_metrics_name)
norm_workload_B = pd.DataFrame(data=Bworkload_knobs_metrics_Norm_id,columns=col_list)



print("--- %s seconds ---" % (time.time() - start_time))
norm_workload_B

--- 0.13962650299072266 seconds ---


,workload id,k1,k2,k3,k4,k5,k6,k7,k8,s1,...,worker_2.Paging_and_Virtual_Memory.pgactivate,worker_2.Paging_and_Virtual_Memory.pgalloc_normal,worker_2.Paging_and_Virtual_Memory.pgfault,worker_2.Paging_and_Virtual_Memory.pgfree,worker_2.Paging_and_Virtual_Memory.pgpgin,worker_2.Paging_and_Virtual_Memory.pgpgout,worker_2.Processes.Blocked,worker_2.Processes.Runnable,worker_2.Processes.fork,worker_2.Processes.pswitch
0,1-1,0.423077,0.176471,1,1,0.0769231,0.919048,1,0.4,0.010101,...,0.187263,0.434244,0.0489238,0.284526,1.30494,0.162126,1.18333,0.223063,0.377736,0.225861
1,1-1,0.192308,0.176471,0.5,0.5,0.0769231,0.919048,1,0.4,0.0909091,...,0.157029,0.369936,0.0443326,0.22023,1.0009,0.172732,0.857143,0.19635,0.317073,0.182314
2,1-1,0.134615,0.0294118,1,1,0.0769231,0.919048,1,0.4,0.0909091,...,0.0767531,0.245833,0.0152169,0.147634,0.861225,0.0733324,0.339815,0.0693224,0.146857,0.13885
3,1-1,0.423077,0.176471,1,1,0.0769231,0.919048,1,0.4,0.0909091,...,0.167855,0.339363,0.0470671,0.228134,0.958487,0.180253,1.68056,0.256706,0.340859,0.198389
4,1-1,0.192308,0.176471,0.5,0.5,0.0769231,0.919048,1,0.4,0.0909091,...,0.158224,0.109943,0.0385281,0.0939092,0.0074373,0.202941,0.0873016,0.201799,0.323645,0.148199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,14-1,0.653846,1,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00161311,0.0686614,0.203777,0.0459204,7.07352e-06,0.0267308,0,0.287024,0.759105,0.0854303
596,14-1,0.192308,0.294118,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00112479,0.021567,0.150025,0.159283,0,0.0119027,0.0208333,0.210566,0.537258,0.0477424
597,14-1,0.269231,0.411765,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00252511,0.0370058,0.141262,0.314006,1.83154e-05,0.0180185,0.0257937,0.259081,0.63011,0.0625165
598,14-1,0.653846,1,0,0,0.0769231,0.919048,0,0.4,0.0909091,...,0.00267613,0.0738297,0.196381,0.514412,1.4147e-05,0.0316406,0.0472222,0.3592,0.795794,0.0872622


In [120]:
start_time = time.time()

#Step2 Split workload_B by creating different dataframes

workload_B_ids = norm_workload_B["workload id"].unique()
workload_B_list = []
for num,workload_B_id in enumerate(workload_B_ids,start=1):
    tmp_workload = norm_workload_B[norm_workload_B['workload id'].isin([workload_B_id])]
    workload_B_list.append(tmp_workload)
    exec("workload_B_%s = tmp_workload"%num)

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.057845354080200195 seconds ---


In [121]:
start_time = time.time()

#Step3 Split each workload into 5 data points and 1 data point as target

num_workloads_B = len(workload_B_list)

workload_B_mapping_data = []
workload_B_target = []

for i in range(num_workloads_B):
    workload_B_mapping_data.append(workload_B_list[i][:-1])
    workload_B_target.append(workload_B_list[i][5:6])
    
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.02693033218383789 seconds ---


In [122]:
start_time = time.time()

#Step4 workload mapping Test for all workload in workload B
all_distance_list = []
for i in range(100):
    tmp_test_workload_B_mapping = workload_B_mapping_data[i]
    tmp_test_workload_B_mapping_cfigs = ((tmp_test_workload_B_mapping)[knobs_list]).values
    
    tmp_distance_list = []
    
    for j in range(58):
        tmp_test_workload_A = workload_A_list[j]
        tmp_test_workload_A_gpr_model = workload_A_GPRmodels_lists[j]   
        tmp_test_workload_A_cfigs = ((tmp_test_workload_A)[knobs_list]).values
        
        tmp_corr_metric_values = []
        
        for k in range(5):
            tmp_data_point = tmp_test_workload_B_mapping_cfigs[k]
            tmp_check = np.where((tmp_data_point==tmp_test_workload_A_cfigs[:,None]).all(-1))
            
            if len(tmp_check[0]) == 0:
                tmp_gpr_predic = []
                for v in range(num_M):
                    tmp_gpr_predic.append(tmp_test_workload_A_gpr_model[v].predict([tmp_data_point])[0])
                tmp_corr_metric_values.append(tmp_gpr_predic)
            
            else:
                tmp_gpr_predic = []
                tmp_row_idx = tmp_check[0][0]
                for n in range(num_M):
                    tmp_gpr_predic.append((((tmp_test_workload_A)[[tmp_centers_metrics_name[n]]]).values)[tmp_row_idx])
                tmp_corr_metric_values.append(tmp_gpr_predic)
        
        #print(tmp_corr_metric_values.shape)
        tmp_test_workload_B_mapping_metrics_norm_value_list = []
        for m in range(num_M):
            tmp_test_workload_B_mapping_metrics_value = tmp_test_workload_B_mapping[[tmp_centers_metrics_name[m]]].values

            tmp_test_workload_B_mapping_metrics_norm_value_list.append(tmp_test_workload_B_mapping_metrics_value)


        tmp_test_workload_B_mapping_metrics_norm_value_arry = np.asarray(tmp_test_workload_B_mapping_metrics_norm_value_list).reshape((num_M,5))       
        tmp_corr_metric_values_arry = np.asarray(tmp_corr_metric_values).reshape((5,num_M)).T

        up_min_list = np.min(tmp_test_workload_B_mapping_metrics_norm_value_arry,axis=0)
        down_min_list = np.min(tmp_corr_metric_values_arry,axis=0)
        tot_min_list = np.min(np.vstack((up_min_list,down_min_list)),axis=0)
        
        up_max_list = np.max(tmp_test_workload_B_mapping_metrics_norm_value_arry,axis=0)
        down_max_list = np.max(tmp_corr_metric_values_arry,axis=0)
        tot_max_list = np.max(np.vstack((up_max_list,down_max_list)),axis=0)

        tmp_test_workload_B_mapping_metrics_norm_value_arry_norm = np.divide((tmp_test_workload_B_mapping_metrics_norm_value_arry - tot_min_list),(tot_max_list - tot_min_list))
        tmp_corr_metric_values_arry_norm = np.divide((tmp_corr_metric_values_arry - tot_min_list),(tot_max_list - tot_min_list))

        tmp_distance = np.sum((tmp_test_workload_B_mapping_metrics_norm_value_arry_norm-tmp_corr_metric_values_arry_norm)*(tmp_test_workload_B_mapping_metrics_norm_value_arry_norm-tmp_corr_metric_values_arry_norm))
        tmp_distance_list.append(tmp_distance)
     
    all_distance_list.append(tmp_distance_list)


    
all_min_idx_list = []
for i in range(100):
    tmp_idx = all_distance_list[i].index(min(all_distance_list[i]))
    all_min_idx_list.append(tmp_idx)

print("--- %s seconds ---" % (time.time() - start_time))


--- 57.54861283302307 seconds ---


In [123]:
start_time = time.time()

#Step5 Concatenate trances from the observed target workload and the nearest neighbor workload for all workload B

predictGPRmodel_list = []
predictScalar_list = []
inputScalar_list = []


for i in range(100):
    tmp_similar_idx = all_min_idx_list[i]
    tmp_similar_workload = workload_A_list[tmp_similar_idx]
    tmp_similar_workload_configs = ((tmp_similar_workload)[knobs_list]).values
    tmp_similar_workload_metrics = ((tmp_similar_workload)[[tmp_centers_metrics_name[0]]]).values
    
    #tmp_workload_B_mapping_metrics_with_similar = ((workload_B_mapping_data[i])[workload_list_centers_metrics_name[tmp_similar_idx]]).values
    #tmp_similar_workload_metrics = ((pruning_workload_list[tmp_similar_idx])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]).values
    #tmp_workload_B_mapping_metrics_with_similar = ((workload_B_mapping_data[i])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]).values
    
    tmp_Augmented_pruned_target_configs = np.array(tmp_similar_workload_configs)
    tmp_Augmented_pruned_target_metrics = np.array(tmp_similar_workload_metrics)
    
    tmp_workload_B_configs = workload_B_mapping_data[i][knobs_list].values
    tmp_workload_B_metrics = workload_B_mapping_data[i][[tmp_centers_metrics_name[0]]].values
    
    for j in range(5):
        tmp_test_workload_B_mapping_cfigs = tmp_workload_B_configs[j]
        tmp_test_workload_B_mapping_metrics_with_similar = tmp_workload_B_metrics[j]
        tmp_check = np.where((tmp_test_workload_B_mapping_cfigs==tmp_similar_workload_configs[:,None]).all(-1))
        
        if len(tmp_check[0]) == 0:
            
            tmp_Augmented_pruned_target_configs = np.append(tmp_Augmented_pruned_target_configs,[tmp_test_workload_B_mapping_cfigs],axis=0)
            tmp_Augmented_pruned_target_metrics = np.append(tmp_Augmented_pruned_target_metrics,[tmp_test_workload_B_mapping_metrics_with_similar],axis=0)
        else:
        
            tmp_row_idx = tmp_check[0][0]
            #print(tmp_row_idx)
            #print('xxxxxxxxxxxxxxxxxxx')
            tmp_Augmented_pruned_target_metrics[tmp_row_idx] = tmp_test_workload_B_mapping_metrics_with_similar
    
    #tmp_metrics_o_size = tmp_Augmented_pruned_target_metrics.T[0].shape[0]
    #tmp_nor_tmp_Augmented_pruned_target_metrics_o = tmp_Augmented_pruned_target_metrics.T[0].reshape((tmp_metrics_o_size,1))
    #print(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    #tmp_o_scalar = MinMaxScaler().fit(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    #predictScalar_list.append(tmp_o_scalar)
    
    #tmp_Augmented_pruned_target_metrics_nomr = tmp_o_scalar.transform(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    
    #tmp_X_scalar = MinMaxScaler().fit(tmp_Augmented_pruned_target_configs)
    #inputScalar_list.append(tmp_X_scalar)
    
    #tmp_Augmented_pruned_target_configs_norm = tmp_X_scalar.transform(tmp_Augmented_pruned_target_configs)
    #print(tmp_Augmented_pruned_target_metrics_nomr)
    
    #print(tmp_Augmented_pruned_target_configs)
    #print('------')
    #print(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    
    test_workload_B_gpr_o = GaussianProcessRegressor(kernel=tmp_kernel).fit(tmp_Augmented_pruned_target_configs, tmp_Augmented_pruned_target_metrics)
    predictGPRmodel_list.append(test_workload_B_gpr_o)
    
print("--- %s seconds ---" % (time.time() - start_time))


--- 13.025176048278809 seconds ---


In [124]:
sca_max = dataScalar.data_max_[12]
sca_min = dataScalar.data_min_[12]

In [125]:
start_time = time.time()

#Step6 predict for all workload B
right_count = 0
for i in range(100):
    tmp_target_config = workload_B_target[i][knobs_list].values
    #print(tmp_target_config)
    #tmp_target_config_norm = inputScalar_list[i].transform(tmp_target_config)
    #print(tmp_target_config_norm)
    tmp_max = dataScalar.data_max_[12]
    tmp_min = dataScalar.data_min_[12]
    tmp_predict = predictGPRmodel_list[i].predict(tmp_target_config)
    tmp_predict_re = tmp_predict * (tmp_max - tmp_min) + tmp_min
    tmp_ground_Truth = workload_B_target[i][[tmp_centers_metrics_name[0]]].values * (tmp_max - tmp_min) + tmp_min
    tmp_diff = np.abs(tmp_predict_re - tmp_ground_Truth)
    print('Predict: ',tmp_predict_re)
    print('Ground Truth: ',tmp_ground_Truth)
    print('diff: ',tmp_diff)
    if (tmp_diff/tmp_ground_Truth) < 0.15:
        right_count += 1
    print('------------')
    
print('Right Predict: ', right_count)
    
print("--- %s seconds ---" % (time.time() - start_time))


Predict:  [[1.54625]]
Ground Truth:  [[7.923000000000002]]
diff:  [[6.376750000000002]]
------------
Predict:  [[261.46899978]]
Ground Truth:  [[260.62625]]
diff:  [[0.8427497813620448]]
------------
Predict:  [[51.17874941]]
Ground Truth:  [[109.0855]]
diff:  [[57.90675059326804]]
------------
Predict:  [[1.54625]]
Ground Truth:  [[61.997]]
diff:  [[60.45075]]
------------
Predict:  [[90.59599385]]
Ground Truth:  [[89.78475]]
diff:  [[0.8112438463530083]]
------------
Predict:  [[27.88949927]]
Ground Truth:  [[24.478500000000007]]
diff:  [[3.4109992717758075]]
------------
Predict:  [[10.36099701]]
Ground Truth:  [[4.5071249999999985]]
diff:  [[5.853872011777263]]
------------
Predict:  [[107.22199738]]
Ground Truth:  [[107.993]]
diff:  [[0.7710026246663517]]
------------
Predict:  [[42.34446075]]
Ground Truth:  [[42.31425000000001]]
diff:  [[0.030210746995344095]]
------------
Predict:  [[51.55149923]]
Ground Truth:  [[52.674]]
diff:  [[1.1225007686333939]]
------------
Predict:  [[1

Predict:  [[18.86374842]]
Ground Truth:  [[10.9345]]
diff:  [[7.929248420327049]]
------------
Predict:  [[214.98074983]]
Ground Truth:  [[196.2085]]
diff:  [[18.772249826920074]]
------------
Predict:  [[6.33574524]]
Ground Truth:  [[3.2158750000000005]]
diff:  [[3.1198702392095905]]
------------
Predict:  [[1.54625]]
Ground Truth:  [[193.85350000000003]]
diff:  [[192.30725000000004]]
------------
Predict:  [[1.54625]]
Ground Truth:  [[94.117]]
diff:  [[92.57075]]
------------
Predict:  [[36.34216439]]
Ground Truth:  [[164.17899999999997]]
diff:  [[127.83683561149844]]
------------
Predict:  [[93.03924681]]
Ground Truth:  [[92.42075]]
diff:  [[0.6184968108380389]]
------------
Predict:  [[51.17874939]]
Ground Truth:  [[41.93025]]
diff:  [[9.248499388925268]]
------------
Predict:  [[1.54625]]
Ground Truth:  [[5.903333333333332]]
diff:  [[4.357083333333333]]
------------
Predict:  [[26.23466599]]
Ground Truth:  [[26.18166666666667]]
diff:  [[0.05299932018628084]]
------------
Predict: 